In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import missingno as msn
import dexplot as dxp
import time
import gc
import numpy as np
from scipy.stats import spearmanr
from datetime import timedelta


In [2]:
import random

In [3]:

# from sklearn.compose import ColumnTransformer
# from sklearn.datasets import fetch_openml
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.model_selection import cross_val_score
# from sklearn.linear_model import Lasso
# from sklearn.preprocessing import MinMaxScaler

# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC, LinearSVC
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.linear_model import Perceptron
# from sklearn.linear_model import SGDClassifier
# from sklearn.tree import DecisionTreeClassifier
# import xgboost as xgb
import lightgbm as lgb

In [4]:
###reading in training set

train=pd.read_csv("training_set_VU_DM.csv")
#test=pd.read_csv("test_set_VU_DM.csv")


In [5]:
# dropping columns with high nulls

to_drop= ['comp1_rate',
'comp1_inv',
'comp1_rate_percent_diff',
'comp2_rate',
'comp2_inv',
'comp2_rate_percent_diff',
'comp3_rate',
'comp3_inv',
'comp3_rate_percent_diff',
'comp4_rate',
'comp4_inv',
'comp4_rate_percent_diff',
'comp5_rate',
'comp5_inv',
'comp5_rate_percent_diff',
'comp6_rate',
'comp6_inv',
'comp6_rate_percent_diff',
'comp7_rate',
'comp7_inv',
'comp7_rate_percent_diff',
'comp8_rate',
'comp8_inv',
'comp8_rate_percent_diff',
'date_time',
'prop_brand_bool',
'gross_bookings_usd',
'visitor_hist_starrating',
'visitor_hist_adr_usd',
'prop_log_historical_price',
'srch_query_affinity_score'
]



#creating dependant variabe 
conditions = [
    (train['booking_bool']> 0),
    (train['click_bool']> 0)
]

choices = [5 ,1]
train['outcome'] = np.select(conditions, choices, default=0)

train = train.drop(to_drop,  axis=1)#.sample(frac=0.2, replace=True, random_state=1)

#filtering out price outliers 
#train = train[train.price_usd < 3001]

In [6]:
def normalize_features(input_df, group_key, target_column, take_log10=False):

    # for numerical stability
    epsilon = 1e-4
    if take_log10:
        input_df[target_column] = np.log10(input_df[target_column] + epsilon)
    methods = ["mean", "std"]

    df = input_df.groupby(group_key).agg({target_column: methods})

    df.columns = df.columns.droplevel()
    col = {}
    for method in methods:
        col[method] = target_column + "_" + method

    df.rename(columns=col, inplace=True)
    df_merge = input_df.merge(df.reset_index(), on=group_key)
    df_merge[target_column + "norm_by" + group_key] = (
        df_merge[target_column] - df_merge[target_column + "_mean"]
    ) / df_merge[target_column + "_std"]
    df_merge = df_merge.drop(labels=[col["mean"], col["std"]], axis=1)

    gc.collect()
    return df_merge


In [7]:
train['price_usd'] =train['price_usd'] / (train['srch_adults_count'] + train['srch_children_count'])

train = normalize_features(train,group_key="srch_id",target_column="price_usd",take_log10=True)
train = normalize_features(train, group_key="prop_id", target_column="price_usd")
#train_reduced = normalize_features(train_reduced, group_key="srch_id", target_column="prop_starrating")

train.head(20)

,srch_id,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_location_score1,prop_location_score2,position,...,srch_children_count,srch_room_count,srch_saturday_night_bool,orig_destination_distance,random_bool,click_bool,booking_bool,outcome,price_usdnorm_bysrch_id,price_usdnorm_byprop_id
0,1,12,187,219,893,3,3.5,2.83,0.0438,27,...,0,1,1,NaN,1,0,0,0,-0.906229,-1.574866
1,498,32,220,219,893,3,3.5,2.83,0.0950,28,...,0,1,0,7500.57,0,0,0,0,-0.336128,0.733022
2,896,5,2,219,893,3,3.5,2.83,0.0438,24,...,0,1,1,NaN,0,0,0,0,0.478557,1.406280
3,1495,5,219,219,893,3,3.5,2.83,0.0438,28,...,0,1,0,NaN,0,0,0,0,0.142423,1.592633
4,2052,5,219,219,893,3,3.5,2.83,0.0438,29,...,0,1,1,77.22,0,0,0,0,0.691599,-0.184266
5,2056,5,219,219,893,3,3.5,2.83,0.0438,30,...,0,1,1,1244.83,0,0,0,0,-0.526815,-0.445164
6,2160,22,92,219,893,3,3.5,2.83,0.0038,25,...,0,1,0,NaN,0,0,0,0,-0.341499,0.088083
7,2521,32,220,219,893,3,3.5,2.83,NaN,9,...,0,1,0,7499.25,1,0,0,0,-0.234918,0.096361
8,2932,5,219,219,893,3,3.5,2.83,0.0950,31,...,0,1,1,8.51,0,0,0,0,-0.324277,-1.296313
9,2937,5,219,219,893,3,3.5,2.83,0.0950,28,...,1,1,1,1980.27,1,0,0,0,0.323049,-0.166934


In [8]:
    ## creating time related features
    
#train['date_time'] = pd.to_datetime(train['date_time'], errors='coerce')

In [9]:
#more time related features

#train['srch_for_date'] = train['date_time']  + pd.to_timedelta(train['srch_booking_window'], unit='D')
#train["srch_for_month"] = train["srch_for_date"].dt.month


In [10]:
#filling in missing values for prop_review_score and prop_starrating using average rating 
#based off similar priced hotels 


train['price_usd'].replace([np.inf, -np.inf], 0, inplace=True)
train['price_usd'] = train['price_usd'].fillna(0)
train['price_bin'] = pd.qcut(train['price_usd'], q=80, precision=0)
train['mean_price_bin_star'] = train.groupby('price_bin')['prop_starrating'].transform('median')
train['mean_price_bin_review'] = train.groupby('price_bin')['prop_review_score'].transform('median')
train['prop_review_score'] = train['prop_review_score'].fillna(0)
train.loc[train.prop_review_score == 0, "prop_review_score"] = train.mean_price_bin_review
train.loc[train.prop_starrating == 0, "prop_starrating"] = train.mean_price_bin_star

train.loc[train['booking_bool']==1,'price_if_booked'] = train['price_usd']
train['med_pp'] = train.groupby('srch_destination_id')['price_if_booked'].transform('median') 
train['loc_%buy'] = train.groupby('srch_destination_id')['booking_bool'].transform('sum') /train.booking_bool.sum()
train['diff_med_pp'] = train['price_usd'] - train['med_pp']
train['diff_med_pp_abs'] = train['diff_med_pp'].abs()
train['price_prev_rank'] = train.groupby("srch_id")["diff_med_pp"].rank("dense", ascending=False)
train['price_prev_abs_rank'] = train.groupby("srch_id")["diff_med_pp_abs"].rank("dense", ascending=False)
train['diff_med_pp'] = train['diff_med_pp']/train['med_pp']
train.head(50)



loc_facts= train[['srch_destination_id', 'med_pp', 'loc_%buy']]
loc_facts= loc_facts.drop_duplicates(subset=['srch_destination_id'])

In [11]:
#comparing price for hotels with other hotels in the same search

train['max_price'] = train.groupby('srch_id')['price_usd'].transform('max') 
train['min_price'] = train.groupby('srch_id')['price_usd'].transform('min') 
train['mean_price'] = train.groupby('srch_id')['price_usd'].transform('median') 
train['diff_max_price'] =(train['max_price'] -train['price_usd']) / train['price_usd']
train['diff_min_price'] = (train['min_price'] - train['price_usd'])/ train['price_usd']
train['diff_min_price'] = train['diff_min_price'].replace(np.nan, 0)
train['diff_mean_price'] = (train['mean_price'] - train['price_usd']) / train['price_usd']
train['price_review_value'] =  train['prop_review_score'] /train['mean_price_bin_review'] 


train['mean_review_srch'] = train.groupby('srch_id')['prop_review_score'].transform('median') 
train['diff_mean_review'] =  (train['prop_review_score'] - train['mean_review_srch']) / train['mean_review_srch']
train['mean_star_srch'] = train.groupby('srch_id')['prop_starrating'].transform('median') 
train['diff_mean_star'] =  (train['prop_starrating'] - train['mean_star_srch']) / train['prop_starrating']
train['mean_location_score'] = train.groupby('srch_id')['prop_location_score1'].transform('median') 
train['diff_location_score'] =  (train['prop_location_score1'] - train['mean_location_score']) / train['prop_location_score1'] 
train['mean_location2_score'] = train.groupby('srch_id')['prop_location_score2'].transform('median')
median = train['prop_location_score2'].median()
train['median_location2_dest_score'] = train.groupby('srch_destination_id')['prop_location_score2'].transform('median') 
#train.loc[train['prop_location_score2'].isnull(), "prop_location_score2"] = train['median_location2_dest_score']
train.loc[train['prop_location_score2'].isnull(), "prop_location_score2"] = -1
train.loc[train['median_location2_dest_score'].isnull(), "median_location2_dest_score"] = 0.007
#train['prop_location_score2'] = train['prop_location_score2'].fillna(value=median, inplace=True )
#train['prop_location_score2'] = train['prop_location_score2'].fillna(0.07,inplace=True)
#train['prop_location_score2'] = train['prop_location_score2'].replace(np.nan, 0)

train['diff_location2_score'] =  (train['prop_location_score2'] - train['median_location2_dest_score']) / train['prop_location_score2']
train.loc[train['diff_location2_score'].isnull(), "diff_location2_score"] = -2

train['median_orig_dest_distance'] = train.groupby('srch_id')['orig_destination_distance'].transform('median')
train['mean_orig_dest_distance'] = train.groupby('srch_id')['orig_destination_distance'].transform('median')
train['diff_orig_dest_distance'] = train['orig_destination_distance'] - train['median_orig_dest_distance']
train['diff_orig_dest_distance1']=train['diff_orig_dest_distance'].abs()
train['rank_distance'] = train.groupby("srch_id")["diff_orig_dest_distance1"].rank("dense", ascending=True)
train.loc[train['rank_distance'].isnull(), 'rank_distance'] = random.choice([7,8,9,10,11,12])
train.loc[train['orig_destination_distance'].isnull(), 'orig_destination_distance'] = train['orig_destination_distance'].median()


train['location_size'] = train.groupby('srch_destination_id')['prop_id'].transform('nunique')
train['location_%'] = train.groupby('srch_destination_id')['srch_id'].transform('nunique') /(train.srch_id.unique()).size

train['srch_results_count'] =  train.groupby('srch_id')['prop_id'].transform('count')
train['srch_%'] =  1/train['srch_results_count']

train['rank_distance_n'] =  train['rank_distance']/train['srch_results_count']

train['rank_price'] = train.groupby("srch_id")["price_usd"].rank("dense", ascending=True)
train['rank_price_n'] = train['rank_price']/train['srch_results_count']

train['rank_location'] = train.groupby("srch_id")["prop_location_score1"].rank("dense", ascending=False)
train['rank_location_n'] = train['rank_location']/train['srch_results_count']

train['rank_review'] = train.groupby("srch_id")["prop_review_score"].rank("dense", ascending=False)
train['rank_review_n'] = train['rank_review']/train['srch_results_count']

train['rank_star'] = train.groupby("srch_id")["prop_starrating"].rank("dense", ascending=False)
train['rank_star_n'] = train['rank_star']/train['srch_results_count']

train['prop_location_score2'] = train['prop_location_score2'].replace(np.nan, 0)

#train_reduced['"prop_location_score2'] = train_reduced['prop_location_score2'].replace(np.nan, 0)
train['rank_location2'] = train.groupby("srch_id")["prop_location_score2"].rank("dense", ascending=False)
train['rank_location2_n'] = train['rank_location2']/train['srch_results_count']

train['rank'] = train['rank_location2']  + train['rank_star'] + train['rank_price']
train['rank_n'] = train['rank_location2_n']  + train['rank_star_n'] + train['rank_price_n']

train['position_rank'] = train.groupby("srch_id")["position"].rank("dense", ascending=True)


In [12]:
# train[['diff_location2_score', 'prop_location_score2','median_location2_dest_score']].isnull().mean() * 100

In [13]:
# getting actual click and booking statistics for each hotel & if there was a booking - how many adults,children,rooms

train['hotel_show_prob'] =  train.groupby('prop_id')['prop_id'].transform('count') / (train.srch_id.unique()).size
train['show_feature']= train['location_%'] *train['hotel_show_prob']
    
train_calcs = train[train.booking_bool == 1]
train_calcs['hotel_avg_children'] = train_calcs.groupby('prop_id')['srch_children_count'].transform('median') 
train_calcs['hotel_avg_adult'] = train_calcs.groupby('prop_id')['srch_adults_count'].transform('median') 
train_calcs['hotel_avg_stay'] = train_calcs.groupby('prop_id')['srch_length_of_stay'].transform('median') 
train_calcs['hotel_avg_room_count'] = train_calcs.groupby('prop_id')['srch_room_count'].transform('median') 
train_calcs['bought'] = train_calcs.groupby('prop_id')['booking_bool'].transform('sum') 
train_calcs = train_calcs[['prop_id','hotel_avg_children', 'hotel_avg_adult', 'hotel_avg_stay','hotel_avg_room_count','bought']]

#pd.crosstab(train.hotel_click_prob, train.prop_id, dropna=False)
train_calcs = train_calcs.drop_duplicates(subset=['prop_id'])

train['hotel_buy_prob'] = train.groupby('prop_id')['booking_bool'].transform('sum') / train.groupby('prop_id')['booking_bool'].transform('count')
train['hotel_buy_share'] = train.groupby('prop_id')['booking_bool'].transform('sum') / train.booking_bool.sum()
train['hotel_click_prob'] = train.groupby('prop_id')['click_bool'].transform('sum') / train.groupby('prop_id')['click_bool'].transform('count')
train['hotel_bought_bool'] = train['hotel_buy_prob'].apply(lambda x: 1 if x > 0 else -10)
train['hotel_click_bool'] = train['hotel_click_prob'].apply(lambda x: 1 if x > 0 else 0)
train['mean_position'] = train.groupby('prop_id')['position_rank'].transform('median') 

hotel_click_stats = train[['prop_id', 'hotel_buy_share','hotel_buy_prob','hotel_click_prob','hotel_bought_bool','hotel_click_bool','mean_position']]

C:\Users\Admin\AppData\Local\Temp\ipykernel_25924\1353321248.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_calcs['hotel_avg_children'] = train_calcs.groupby('prop_id')['srch_children_count'].transform('median')
C:\Users\Admin\AppData\Local\Temp\ipykernel_25924\1353321248.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_calcs['hotel_avg_adult'] = train_calcs.groupby('prop_id')['srch_adults_count'].transform('median')
C:\Users\Admin\AppData\Local\Temp\ipykernel_25924\1353321248.py:9: Sett

In [14]:
train_reduced = train
### merging hotel specific cals to main dataset 
train_reduced = train_reduced.merge(train_calcs, left_on=['prop_id'], right_on=['prop_id'], how='left')

train_reduced['srch_mean_avg_children'] =  train_reduced.groupby('srch_id')['hotel_avg_children'].transform('median') 
train_reduced['srch_mean_avg_children'] = train_reduced['srch_mean_avg_children'].replace(np.nan,  0)
train_reduced['srch_mean_avg_adult'] =  train_reduced.groupby('srch_id')['hotel_avg_adult'].transform('median')
train_reduced['srch_mean_avg_adult'] = train_reduced['srch_mean_avg_adult'].replace(np.nan,  0)
train_reduced['srch_mean_avg_stay'] =  train_reduced.groupby('srch_id')['hotel_avg_stay'].transform('median')
train_reduced['srch_mean_avg_stay'] = train_reduced['srch_mean_avg_stay'].replace(np.nan,  0)
train_reduced['srch_mean_avg_room_count'] =  train_reduced.groupby('srch_id')['hotel_avg_room_count'].transform('median')
train_reduced['srch_mean_avg_room_count'] = train_reduced['srch_mean_avg_room_count'].replace(np.nan,  0)


train_reduced.loc[train_reduced['hotel_avg_children'].isnull(), "hotel_avg_children"] = train_reduced['srch_mean_avg_children']
train_reduced.loc[train_reduced['hotel_avg_adult'].isnull(), "hotel_avg_adult"] = train_reduced['srch_mean_avg_adult']
train_reduced.loc[train_reduced['hotel_avg_stay'].isnull(), "hotel_avg_stay"] = train_reduced['srch_mean_avg_stay']
train_reduced.loc[train_reduced['hotel_avg_room_count'].isnull(), "hotel_avg_room_count"] = train_reduced['srch_mean_avg_room_count']

train_reduced['diff_avg_children'] = (train_reduced['srch_children_count'] - train_reduced['hotel_avg_children'])
train_reduced['diff_avg_adult'] = (train_reduced['srch_adults_count'] - train_reduced['hotel_avg_adult'])/train_reduced['srch_adults_count']
train_reduced['diff_avg_stay'] = (train_reduced['srch_length_of_stay'] - train_reduced['hotel_avg_stay'])/train_reduced['srch_length_of_stay']
train_reduced['diff_avg_room_count'] = (train_reduced['srch_room_count'] - train_reduced['hotel_avg_room_count'])/train_reduced['srch_room_count']

train_reduced['diff_avg_children1']=train_reduced['diff_avg_children'].abs()
train_reduced['diff_avg_adult1']=train_reduced['diff_avg_adult'].abs()
train_reduced['diff_avg_stay1']=train_reduced['diff_avg_stay'].abs()
train_reduced['diff_avg_room_count1']=train_reduced['diff_avg_room_count'].abs()

train_reduced['mean_position_rank'] = train_reduced.groupby("srch_id")['mean_position'].rank("dense", ascending=True)
train_reduced['mean_position_rank_n'] = train_reduced['mean_position_rank']/train_reduced['srch_results_count']


train_reduced['rank_child_match'] = train_reduced.groupby("srch_id")["diff_avg_children1"].rank("dense", ascending=True)
train_reduced['rank_child_match_n'] = train_reduced['rank_child_match']/train_reduced['srch_results_count']

train_reduced['rank_adult_match'] = train_reduced.groupby("srch_id")['diff_avg_adult1'].rank("dense", ascending=True)
train_reduced['rank_adult_match_n'] = train_reduced['rank_adult_match']/train_reduced['srch_results_count']

train_reduced['rank_stay_match'] = train_reduced.groupby("srch_id")['diff_avg_stay1'].rank("dense", ascending=True)
train_reduced['rank_stay_match_n'] = train_reduced['rank_stay_match']/train_reduced['srch_results_count']

train_reduced['rank_room_match'] = train_reduced.groupby("srch_id")['diff_avg_room_count1'].rank("dense", ascending=True)
train_reduced['rank_room_match_n'] = train_reduced['rank_room_match']/train_reduced['srch_results_count']

train_reduced['rank2'] = train_reduced['rank_child_match'] + train_reduced['rank_adult_match'] + train_reduced['rank_stay_match'] + train_reduced['rank_room_match']
train_reduced['rank2_n'] = train_reduced['rank_child_match_n'] + train_reduced['rank_adult_match_n'] + train_reduced['rank_stay_match_n'] + train_reduced['rank_room_match_n']

train_reduced['rank3'] = train_reduced['rank_distance'] + train_reduced['rank_price'] + train_reduced['rank_stay_match'] + train_reduced['mean_position_rank']
train_reduced['rank3_n'] = train_reduced['rank_distance_n'] + train_reduced['rank_price_n'] + train_reduced['rank_stay_match_n'] + train_reduced['mean_position_rank_n']

train_reduced['rank5'] = train_reduced['rank_child_match'] + train_reduced['rank_adult_match'] + train_reduced['rank_stay_match'] + train_reduced['rank_room_match'] +train_reduced['price_prev_rank']

train_reduced['rank4'] = train_reduced['rank2'] + train['rank'] + train_reduced['rank_distance'] + train_reduced['mean_position_rank'] + train_reduced['price_prev_abs_rank']
#train_reduced['estimated_position'] = train_reduced.groupby("prop_id")["position"].transform('mean')

#to_keep[['estimated_position','prop_id','hotel_click_bool','hotel_book_bool','location_clicks%','location_booking%']] = train_reduced[['estimated_position','prop_id','hotel_click_bool','hotel_book_bool','location_clicks%','location_booking%']]

In [15]:
train_reduced.head(20)

,srch_id,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_location_score1,prop_location_score2,position,...,rank_stay_match,rank_stay_match_n,rank_room_match,rank_room_match_n,rank2,rank2_n,rank3,rank3_n,rank5,rank4
0,1,12,187,219,893,3,3.5,2.83,0.0438,27,...,1.0,0.035714,1.0,0.035714,5.0,0.178571,29.0,1.035714,25.0,49.0
1,498,32,220,219,893,3,3.5,2.83,0.0950,28,...,1.0,0.028571,1.0,0.028571,6.0,0.171429,39.0,1.114286,19.0,72.0
2,896,5,2,219,893,3,3.5,2.83,0.0438,24,...,1.0,0.027778,1.0,0.027778,6.0,0.166667,50.0,1.388889,15.0,80.0
3,1495,5,219,219,893,3,3.5,2.83,0.0438,28,...,1.0,0.029412,1.0,0.029412,6.0,0.176471,44.0,1.294118,20.0,79.0
4,2052,5,219,219,893,3,3.5,2.83,0.0438,29,...,1.0,0.029412,1.0,0.029412,4.0,0.117647,44.0,1.294118,10.0,79.0
5,2056,5,219,219,893,3,3.5,2.83,0.0438,30,...,1.0,0.028571,1.0,0.028571,4.0,0.114286,50.0,1.428571,19.0,78.0
6,2160,22,92,219,893,3,3.5,2.83,0.0038,25,...,4.0,0.148148,1.0,0.037037,7.0,0.259259,41.0,1.518519,25.0,76.0
7,2521,32,220,219,893,3,3.5,2.83,-1.0000,9,...,3.0,0.088235,1.0,0.029412,6.0,0.176471,50.0,1.470588,22.0,68.0
8,2932,5,219,219,893,3,3.5,2.83,0.0950,31,...,3.0,0.085714,1.0,0.028571,6.0,0.171429,38.0,1.085714,20.0,66.0
9,2937,5,219,219,893,3,3.5,2.83,0.0950,28,...,4.0,0.117647,1.0,0.029412,11.0,0.323529,42.0,1.235294,22.0,79.0


In [16]:
# train_reduced[['rank2_n','rank_n','rank_distance_n','mean_position_rank_n']].head(50)

In [17]:
del train

In [18]:
###train_reduced['prop_location_score2'].count()

In [19]:
#del df

In [20]:
#dropped price bin
train_reduced = train_reduced.drop(['price_bin'], axis = 1)
#train_reduced.to_csv('hotel_calcs2.csv')

In [21]:
#del train1

In [22]:
##droppping columns not needed 

# to_drop = ['site_id', 'date_time', 'visitor_location_country_id', 'prop_country_id', 'prop_brand_bool', 'orig_destination_distance', 'click_bool',  'booking_bool', 'prop_location_score2'
#           ,'gross_bookings_usd', 'price_usd', 'max_price','min_price', 'mean_price', 'mean_price_bin_review','mean_price_bin_star', 'bought'] 

to_drop = ['site_id', 'visitor_location_country_id', 'prop_country_id',  'orig_destination_distance', 'click_bool',  'booking_bool'
          , 'max_price','min_price', 'mean_price', 'mean_price_bin_review','mean_price_bin_star', 'bought','position'] 



           
train1 = train_reduced.drop(to_drop,  axis=1)

#train1.to_csv('training_data.csv')


In [23]:
#train_reduced['rank4']

In [24]:
#train1=pd.read_csv('training_data.csv')

In [25]:
#train1.columns


In [26]:

##changing nan or null values to 0
#train1.replace([np.inf, -np.inf], 0, inplace=True)

#train1=train1.dropna(axis=1,how="any")

In [27]:
##ordering columns and selecting predictors + outcome
#train1 = train1[['srch_id', 'srch_destination_id', 'prop_id', 'prop_starrating', 'prop_location_score1','prop_log_historical_price','promotion_flag','srch_length_of_stay','srch_booking_window','srch_adults_count','srch_children_count','srch_room_count','srch_saturday_night_bool','diff_max_price','diff_min_price','diff_mean_price','hotel_show_prob', 'prop_review_score','price_review_value','rank_child_match','rank_adult_match','rank_stay_match','rank_room_match','random_bool','mean_review_srch','diff_mean_review','mean_star_srch','diff_mean_star','rank_price','rank_location','rank_review','rank_star','rank_location2','rank', 'position','rank_distance','rank2', 'rank3','location_size','outcome']]
# train1 = train1[['srch_id', 'prop_id', 'prop_starrating', 'prop_review_score',
#        'prop_location_score1', 'promotion_flag',
#        'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
#        'srch_adults_count', 'srch_children_count', 'srch_room_count',
#        'srch_saturday_night_bool', 'random_bool',  'diff_max_price',
#        'diff_min_price', 'diff_mean_price', 'price_review_value',
#        'mean_review_srch', 'diff_mean_review', 'mean_star_srch',
#        'diff_mean_star', 'mean_location_score', 'median_location2_dest_score',
#        'rank_distance', 'location_size', 'location_%', 'srch_results_count',
#        'srch_%', 'rank_distance_n', 'rank_price', 'rank_price_n',
#        'rank_location', 'rank_location_n', 'rank_review', 'rank_review_n',
#        'rank_star', 'rank_star_n', 'rank_location2', 'rank_location2_n',
#        'rank', 'rank_n', 'hotel_show_prob', 'show_feature', 'hotel_bought_bool',
#        'hotel_click_bool', 'mean_position', 'hotel_avg_children',
#        'hotel_avg_adult', 'hotel_avg_stay', 'hotel_avg_room_count',
#        'srch_mean_avg_children', 'srch_mean_avg_adult', 'srch_mean_avg_stay',
#        'srch_mean_avg_room_count', 'diff_avg_children', 'diff_avg_adult',
#        'diff_avg_stay', 'diff_avg_room_count', 
#        'rank_child_match', 'rank_child_match_n', 'rank_adult_match',
#        'rank_adult_match_n', 'rank_stay_match', 'rank_stay_match_n',
#        'rank_room_match', 'rank_room_match_n', 'rank2', 'rank2_n', 'rank3',
#        'rank3_n', 'price_usdnorm_bysrch_id','mean_position_rank','mean_position_rank_n','rank4_n','outcome']]

train1 = train1[['srch_id', 'prop_id', 'promotion_flag',
     'random_bool',  'diff_max_price',
       'diff_mean_price',
       'diff_mean_review',
       'diff_mean_star',  
       'location_size',  'srch_results_count','prop_starrating',
       'srch_%','rank_distance','rank_price',
       'rank_location', 'rank_review',
          'rank_star', 'rank_location2', 'prop_location_score2',
      'rank' ,'hotel_show_prob', 'show_feature', 'hotel_bought_bool',
         'mean_position', 'hotel_buy_prob',
         'rank_child_match', 'rank_adult_match', 'rank_stay_match',

         'rank_room_match', 'rank2',
       'rank3', 'price_usdnorm_bysrch_id','mean_position_rank_n','mean_position_rank','diff_med_pp','price_prev_rank','price_prev_abs_rank','rank4', 'rank5','hotel_buy_share','price_usdnorm_byprop_id', 'loc_%buy','outcome']]



In [28]:
#train1.to_csv('test_test.csv')

In [29]:
del train_reduced

In [30]:
df = train1[train1.outcome >= 1]
df1 = train1[train1.outcome == 0]
df1 = df1.sample(n=len(df)*15)

frames = [df, df1]

train1 = pd.concat(frames)

In [31]:
train1.head(50)

,srch_id,prop_id,promotion_flag,random_bool,diff_max_price,diff_mean_price,diff_mean_review,diff_mean_star,location_size,srch_results_count,...,mean_position_rank,diff_med_pp,price_prev_rank,price_prev_abs_rank,rank4,rank5,hotel_buy_share,price_usdnorm_byprop_id,loc_%buy,outcome
27,10657,893,0,1,0.077971,0.001849,0.000000,0.000000,91,30,...,14.0,0.038490,13.0,21.0,75.0,19.0,0.000072,-0.163142,0.001684,5
63,28204,893,1,0,0.069397,0.041321,0.000000,0.000000,519,29,...,17.0,0.141235,17.0,17.0,70.0,25.0,0.000072,0.887423,0.002616,1
67,30410,893,0,1,0.275062,-0.004272,0.000000,0.000000,91,35,...,17.0,0.285468,15.0,15.0,69.0,21.0,0.000072,1.646477,0.001684,1
70,31467,893,0,0,0.107016,-0.037997,-0.125000,0.000000,519,31,...,15.0,0.080174,10.0,15.0,71.0,14.0,0.000072,0.424093,0.002616,5
81,36093,893,0,0,0.165200,0.078137,-0.125000,0.000000,519,14,...,9.0,0.161620,14.0,14.0,41.0,21.0,0.000072,1.042101,0.002616,5
120,53860,893,0,1,0.069817,-0.028758,-0.125000,0.000000,91,29,...,16.0,0.243052,7.0,7.0,69.0,13.0,0.000072,1.335698,0.001684,1
171,80106,893,0,0,0.059182,0.000000,-0.125000,0.000000,630,5,...,4.0,0.008207,3.0,5.0,28.0,7.0,0.000072,0.105948,0.004068,5
174,83132,893,0,0,0.137211,0.002629,-0.066667,0.000000,91,34,...,17.0,0.246991,12.0,12.0,75.0,18.0,0.000072,1.364554,0.001684,5
284,148843,893,1,0,0.133889,0.046970,-0.125000,0.000000,519,33,...,17.0,0.226351,21.0,21.0,74.0,29.0,0.000072,1.533278,0.002616,5
330,172572,893,1,0,0.426942,0.171121,-0.125000,-0.333333,519,34,...,16.0,-0.127330,31.0,9.0,59.0,43.0,0.000072,-1.150446,0.002616,5


In [32]:
train1 = train1.sort_values(['srch_id', 'prop_id'],ascending = [True, False])

In [33]:
from sklearn.model_selection import GroupShuffleSplit

def remove_columns(x1, ignore_column=["srch_id", "prop_id", "position", 'outcome']):
    ignore_column = [c for c in ignore_column if c in x1.columns.values]
    # print('Dropping columns: {}'.format(ignore_column))
    # ignore_column_numbers = [x1.columns.get_loc(x) for x in ignore_column]
    x1 = x1.drop(labels=ignore_column, axis=1)
    # print('Columns after dropping: {}'.format(x1.columns.values))
    return x1

def input_estimated_position(training_data, srch_id_dest_id_dict):
    training_data = training_data.merge(
        srch_id_dest_id_dict, how="left", on=["srch_destination_id", "prop_id"]
    )
    print(training_data.head())
    return training_data


def split_train_data(data_for_training):
    
    gss = GroupShuffleSplit(test_size=.3, n_splits=1, random_state = 7).split(data_for_training, groups=train1['srch_id'])

    X_train_inds, X_test_inds = next(gss)

    train_data= data_for_training.iloc[X_train_inds]
    x1 = train_data.loc[:, ~train_data.columns.isin(['id','outcome'])]
    y1 = train_data.loc[:, train_data.columns.isin(['outcome'])]

    test_data= data_for_training.iloc[X_test_inds]

    #We need to keep the id for later predictions
    x2 = test_data.loc[:, ~test_data.columns.isin(['outcome'])]
    y2 = test_data.loc[:, test_data.columns.isin(['outcome'])]


    srch_id_dest_id_dict = x1.loc[x1["random_bool"] == 0]

    # estimated position calculation
#     srch_id_dest_id_dict = x1.loc[x1["random_bool"] == 0]
#     srch_id_dest_id_dict = x1.groupby(["srch_destination_id", "prop_id"]).agg(
#         {"position": "mean"}
#     )
#     srch_id_dest_id_dict = srch_id_dest_id_dict.rename(
#         index=str, columns={"position": "estimated_position"}
#     ).reset_index()
#     srch_id_dest_id_dict["srch_destination_id"] = (
#         srch_id_dest_id_dict["srch_destination_id"].astype(str).astype(int)
#     )
#     srch_id_dest_id_dict["prop_id"] = (
#         srch_id_dest_id_dict["prop_id"].astype(str).astype(int)
#     )
#     srch_id_dest_id_dict["estimated_position"] = (
#         1 / srch_id_dest_id_dict["estimated_position"]
#     )
#     x1 = input_estimated_position(x1, srch_id_dest_id_dict)
#     x2 = input_estimated_position(x2, srch_id_dest_id_dict)

    groups = x1["srch_id"].value_counts(sort=False).sort_index()
    eval_groups = x2["srch_id"].value_counts(sort=False).sort_index()
    len(eval_groups), len(x2), len(x1), len(groups)

    x1 = remove_columns(x1)
    x2 = remove_columns(x2)
    return (x1, x2, y1, y2, groups, eval_groups, srch_id_dest_id_dict)

method = "dart"
lr = 0.12
x1, x2, y1, y2, groups, eval_groups, srch_id_dest_id_dict = split_train_data(train1)

In [34]:
#del train_data
#del data_for_training 
del srch_id_dest_id_dict

In [35]:
%%time
name_of_model=None

if not name_of_model:
    name_of_model = str(int(time.time()))

clf = lgb.LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    n_estimators=300,
    learning_rate=lr,
    max_position=5,
    label_gain=[0, 1,2,3,4,5, 6],
    random_state=69,
    seed=69,
    boosting=method,)
gc.collect()

print("Training on train set with columns: {}".format(x1.columns.values))
model = clf.fit(
    x1,
    y1,
    eval_set=[(x1, y1), (x2, y2)],
    eval_group=[groups, eval_groups],
    group=groups,
    eval_at=5,
    verbose=20,
    early_stopping_rounds=200,
    #categorical_feature=categorical_features_numbers,
)
    
print('done')
#print(model)
#gc.collect()
#pickle.dump(clf, open(os.path.join(output_dir, "model.dat"), "wb"))
#pickle.dump(model, open(os.path("model.dat", "wb")))



Training on train set with columns: ['promotion_flag' 'random_bool' 'diff_max_price' 'diff_mean_price'
 'diff_mean_review' 'diff_mean_star' 'location_size' 'srch_results_count'
 'prop_starrating' 'srch_%' 'rank_distance' 'rank_price' 'rank_location'
 'rank_review' 'rank_star' 'rank_location2' 'prop_location_score2' 'rank'
 'hotel_show_prob' 'show_feature' 'hotel_bought_bool' 'mean_position'
 'hotel_buy_prob' 'rank_child_match' 'rank_adult_match' 'rank_stay_match'
 'rank_room_match' 'rank2' 'rank3' 'price_usdnorm_bysrch_id'
 'mean_position_rank_n' 'mean_position_rank' 'diff_med_pp'
 'price_prev_rank' 'price_prev_abs_rank' 'rank4' 'rank5' 'hotel_buy_share'
 'price_usdnorm_byprop_id' 'loc_%buy']


C:\Users\Admin\Documents\Masters College\datamining\assignment2\.venv\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Admin\Documents\Masters College\datamining\assignment2\.venv\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Unknown parameter: max_position
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] seed is set=69, random_state=69 will be ignored. Current value: seed=69


C:\Users\Admin\Documents\Masters College\datamining\assignment2\.venv\lib\site-packages\lightgbm\callback.py:223: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


[20]	training's ndcg@5: 0.567838	valid_1's ndcg@5: 0.560882
[40]	training's ndcg@5: 0.573177	valid_1's ndcg@5: 0.565109
[60]	training's ndcg@5: 0.57755	valid_1's ndcg@5: 0.568011
[80]	training's ndcg@5: 0.579244	valid_1's ndcg@5: 0.569835
[100]	training's ndcg@5: 0.58406	valid_1's ndcg@5: 0.573994
[120]	training's ndcg@5: 0.587891	valid_1's ndcg@5: 0.577418
[140]	training's ndcg@5: 0.589398	valid_1's ndcg@5: 0.578613
[160]	training's ndcg@5: 0.590073	valid_1's ndcg@5: 0.579064
[180]	training's ndcg@5: 0.590939	valid_1's ndcg@5: 0.579132
[200]	training's ndcg@5: 0.592	valid_1's ndcg@5: 0.580036
[220]	training's ndcg@5: 0.592646	valid_1's ndcg@5: 0.580589
[240]	training's ndcg@5: 0.594194	valid_1's ndcg@5: 0.581496
[260]	training's ndcg@5: 0.595045	valid_1's ndcg@5: 0.581842
[280]	training's ndcg@5: 0.595499	valid_1's ndcg@5: 0.581965
[300]	training's ndcg@5: 0.59662	valid_1's ndcg@5: 0.582424
done
CPU times: total: 58min 27s
Wall time: 8min 24s


In [36]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

# # sorted(zip(clf.feature_importances_, X.columns), reverse=True)
# feature_imp = pd.DataFrame(sorted(zip(model.feature_importances_,x2.columns)), columns=['Value','Feature'])

# plt.figure(figsize=(20, 10))
# sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
# plt.title('LightGBM Features (avg over folds)')
# plt.tight_layout()
# plt.show()
# plt.savefig('lgbm_importances-01.png')

In [37]:
# # writing the results to a dataframe to analyse seperatly if needed ( ie how many false negatives, check values ect)

# result = pd.DataFrame(y1, columns=['actual'])
# predicted= pd.DataFrame(y2, columns=['predicted'])

# df = pd.DataFrame(X_test)
# df['result'] = result
# df['perdicted'] = predicted
# df.to_csv('results2.csv')
# df.head()

In [38]:
###################### pred 
del x1,y1,x2,y2


In [39]:
del train1

In [40]:
#### import competition dataset 

to_pred=pd.read_csv("test_set_VU_DM.csv")

to_drop= ['comp1_rate',
'comp1_inv',
'comp1_rate_percent_diff',
'comp2_rate',
'comp2_inv',
'comp2_rate_percent_diff',
'comp3_rate',
'comp3_inv',
'comp3_rate_percent_diff',
'comp4_rate',
'comp4_inv',
'comp4_rate_percent_diff',
'comp5_rate',
'comp5_inv',
'comp5_rate_percent_diff',
'comp6_rate',
'comp6_inv',
'comp6_rate_percent_diff',
'comp7_rate',
'comp7_inv',
'comp7_rate_percent_diff',
'comp8_rate',
'comp8_inv',
'comp8_rate_percent_diff',
'date_time',
'prop_brand_bool',
'visitor_hist_starrating',
'visitor_hist_adr_usd',
'prop_log_historical_price',
'srch_query_affinity_score'
]

#train['outcome'] = pd.Series(train['booking_bool']> 0, 500 + train['position'] , train['click_bool']> 0, 100 +train['position'] , 0 + train['position'] )



to_pred = to_pred.drop(to_drop,  axis=1)#.sample(frac=0.2, replace=True, random_state=1)
#test_reduced  = test.iloc[: , :N].sample(frac=0.33, replace=True, random_state=1)


In [41]:
# creating same features as test data

#to_pred['date_time'] = pd.to_datetime(to_pred['date_time'], errors='coerce')

##################
to_pred = normalize_features(to_pred,group_key="srch_id",target_column="price_usd",take_log10=True)
to_pred = normalize_features(to_pred, group_key="prop_id", target_column="price_usd")



# to_pred['srch_for_date'] = to_pred['date_time']  + pd.to_timedelta(to_pred['srch_booking_window'], unit='D')
# to_pred["srch_for_month"] = to_pred["srch_for_date"].dt.month

#train['srch_for_date'] = train['date_time']  + pd.to_timedelta(train['srch_booking_window'], unit='D')
#train["srch_for_month"] = train["srch_for_date"].dt.month

####################
to_pred['price_usd'] =to_pred['price_usd'] / (to_pred['srch_adults_count'] + to_pred['srch_children_count'])
to_pred['price_usd'].replace([np.inf, -np.inf], 0, inplace=True)
to_pred['price_usd'] = to_pred['price_usd'].fillna(0)
to_pred['price_bin'] = pd.qcut(to_pred['price_usd'], q=80, precision=0)
to_pred['mean_price_bin_star'] = to_pred.groupby('price_bin')['prop_starrating'].transform('mean')
to_pred['mean_price_bin_review'] = to_pred.groupby('price_bin')['prop_review_score'].transform('mean')
to_pred['prop_review_score'] = to_pred['prop_review_score'].fillna(0)
to_pred.loc[to_pred.prop_review_score == 0, "prop_review_score"] = to_pred.mean_price_bin_review
to_pred.loc[to_pred.prop_starrating == 0, "prop_starrating"] = to_pred.mean_price_bin_star

#############
to_pred = to_pred.merge(loc_facts, left_on=['srch_destination_id'], right_on=['srch_destination_id'], how='left')

to_pred['diff_med_pp'] = to_pred['price_usd'] - to_pred['med_pp']
to_pred['diff_med_pp_abs'] = to_pred['diff_med_pp'].abs()
to_pred['price_prev_rank'] = to_pred.groupby("srch_id")["diff_med_pp"].rank("dense", ascending=False)
to_pred['price_prev_abs_rank'] = to_pred.groupby("srch_id")["diff_med_pp_abs"].rank("dense", ascending=False)
to_pred['diff_med_pp'] = to_pred['diff_med_pp']/to_pred['med_pp']
#to_pred.head(50)

##################

to_pred['max_price'] = to_pred.groupby('srch_id')['price_usd'].transform('max') 
to_pred['min_price'] = to_pred.groupby('srch_id')['price_usd'].transform('min') 
to_pred['mean_price'] = to_pred.groupby('srch_id')['price_usd'].transform('mean') 
to_pred['diff_max_price'] =(to_pred['max_price'] -to_pred['price_usd']) / to_pred['price_usd']
to_pred['diff_min_price'] = (to_pred['min_price'] - to_pred['price_usd'])/ to_pred['price_usd']
to_pred['diff_min_price'] = to_pred['diff_min_price'].replace(np.nan, 0)
to_pred['diff_mean_price'] = (to_pred['mean_price'] - to_pred['price_usd']) / to_pred['price_usd']
to_pred['price_review_value'] =  to_pred['prop_review_score'] /to_pred['mean_price_bin_review'] 


to_pred['mean_review_srch'] = to_pred.groupby('srch_id')['prop_review_score'].transform('mean') 
to_pred['diff_mean_review'] =  (to_pred['prop_review_score'] - to_pred['mean_review_srch']) / to_pred['mean_review_srch']
to_pred['mean_star_srch'] = to_pred.groupby('srch_id')['prop_starrating'].transform('mean') 
to_pred['diff_mean_star'] =  (to_pred['prop_starrating'] - to_pred['mean_star_srch']) / to_pred['prop_starrating']
to_pred['mean_location_score'] = to_pred.groupby('srch_id')['prop_location_score1'].transform('mean') 
to_pred['diff_location_score'] =  (to_pred['prop_location_score1'] - to_pred['mean_location_score']) / to_pred['prop_location_score1']


to_pred['mean_location2_score'] = to_pred.groupby('srch_id')['prop_location_score2'].transform('mean')
median = to_pred['prop_location_score2'].median()
to_pred['median_location2_dest_score'] = to_pred.groupby('srch_destination_id')['prop_location_score2'].transform('median') 
to_pred.loc[to_pred['prop_location_score2'].isnull(), "prop_location_score2"] = -2
to_pred.loc[to_pred['median_location2_dest_score'].isnull(), "median_location2_dest_score"] = 0.007
#train['prop_location_score2'] = train['prop_location_score2'].fillna(value=median, inplace=True )
#train['prop_location_score2'] = train['prop_location_score2'].fillna(0.07,inplace=True)
#train['prop_location_score2'] = train['prop_location_score2'].replace(np.nan, 0)

to_pred['diff_location2_score'] =  (to_pred['prop_location_score2'] - to_pred['median_location2_dest_score']) / to_pred['prop_location_score2']
to_pred.loc[to_pred['diff_location2_score'].isnull(), "diff_location2_score"] = 0



to_pred['median_orig_dest_distance'] = to_pred.groupby('srch_id')['orig_destination_distance'].transform('median')
to_pred['mean_orig_dest_distance'] = to_pred.groupby('srch_id')['orig_destination_distance'].transform('mean')
to_pred['diff_orig_dest_distance'] = to_pred['orig_destination_distance'] - to_pred['median_orig_dest_distance']
to_pred['diff_orig_dest_distance1']=to_pred['diff_orig_dest_distance'].abs()
to_pred['rank_distance'] = to_pred.groupby("srch_id")["diff_orig_dest_distance1"].rank("dense", ascending=True)
to_pred.loc[to_pred['orig_destination_distance'].isnull(), 'orig_destination_distance'] = to_pred['orig_destination_distance'].mean()
to_pred.loc[to_pred['rank_distance'].isnull(), 'rank_distance'] = random.choice([7,8,9,10,11,12])



to_pred['location_size'] = to_pred.groupby('srch_destination_id')['prop_id'].transform('nunique')
to_pred['location_%'] = to_pred.groupby('srch_destination_id')['srch_id'].transform('nunique') /(to_pred.srch_id.unique()).size

to_pred['srch_results_count'] =  to_pred.groupby('srch_id')['prop_id'].transform('count')
to_pred['srch_%'] =  1/to_pred['srch_results_count']

to_pred['rank_distance_n'] =  to_pred['rank_distance']/to_pred['srch_results_count']

to_pred['rank_price'] = to_pred.groupby("srch_id")["price_usd"].rank("dense", ascending=True)
to_pred['rank_price_n'] = to_pred['rank_price']/to_pred['srch_results_count']

to_pred['rank_location'] = to_pred.groupby("srch_id")["prop_location_score1"].rank("dense", ascending=False)
to_pred['rank_location_n'] = to_pred['rank_location']/to_pred['srch_results_count']

to_pred['rank_review'] = to_pred.groupby("srch_id")["prop_review_score"].rank("dense", ascending=False)
to_pred['rank_review_n'] = to_pred['rank_review']/to_pred['srch_results_count']

to_pred['rank_star'] = to_pred.groupby("srch_id")["prop_starrating"].rank("dense", ascending=False)
to_pred['rank_star_n'] = to_pred['rank_star']/to_pred['srch_results_count']

to_pred['prop_location_score2'] = to_pred['prop_location_score2'].replace(np.nan, 0)

#train_reduced['"prop_location_score2'] = train_reduced['prop_location_score2'].replace(np.nan, 0)
to_pred['rank_location2'] = to_pred.groupby("srch_id")["prop_location_score2"].rank("dense", ascending=False)
to_pred['rank_location2_n'] = to_pred['rank_location2']/to_pred['srch_results_count']

to_pred['rank'] = to_pred['rank_location2']  + to_pred['rank_star'] + to_pred['rank_price']
to_pred['rank_n'] = to_pred['rank_location2_n']  + to_pred['rank_star_n'] + to_pred['rank_price_n']



#################

to_pred['hotel_show_prob'] =  to_pred.groupby('prop_id')['prop_id'].transform('count') / (to_pred.srch_id.unique()).size
to_pred['show_feature']= to_pred['location_%'] *to_pred['hotel_show_prob']

In [42]:
### mergeing hotel specific stats from test dataset 

to_pred_reduced = to_pred.merge(train_calcs, left_on=['prop_id'], right_on=['prop_id'], how='left')
hotel_click_stats = hotel_click_stats.drop_duplicates(subset=['prop_id'])
to_pred_reduced = to_pred_reduced.merge(hotel_click_stats, left_on=['prop_id'], right_on=['prop_id'], how='left')

In [43]:
del to_pred
del hotel_click_stats
del train_calcs

In [44]:

# #to_pred_reduced .columns

# to_pred_reduced[['hotel_buy_prob', 'hotel_click_prob','hotel_bought_bool' ,'hotel_click_bool' ,'mean_position']].isnull().mean() * 100

In [45]:
to_pred_reduced['hotel_buy_prob'] = to_pred_reduced['hotel_buy_prob'].replace(np.nan,  0)
to_pred_reduced['hotel_click_prob'] = to_pred_reduced['hotel_click_prob'].replace(np.nan,  0)
to_pred_reduced['hotel_bought_bool'] = to_pred_reduced['hotel_bought_bool'].replace(np.nan, to_pred_reduced['hotel_bought_bool'].mean())
to_pred_reduced['hotel_click_bool'] = to_pred_reduced['hotel_click_bool'].replace(np.nan,  to_pred_reduced['hotel_click_bool'].mean())
#to_pred_reduced['mean_position'] = to_pred_reduced['mean_position'].replace(np.nan,  random.choice([7,8,9,10,11,12,13,14,15,16]))

In [46]:


#train_reduced = train_reduced.merge(train_clacs, on='prop_id', how='right', suffixes=('_1', '_2'))
#to_pred_reduced  = pd.concat([to_pred_reduced.set_index('prop_id'),train_calcs.set_index('prop_id')], axis=1, join='outer').reset_index()
#to_pred_reduced2  = pd.concat([to_pred_reduced.set_index('prop_id'),hotel_click_stats.set_index('prop_id')], axis=1, join='outer').reset_index()
to_pred_reduced.loc[to_pred_reduced['mean_position'].isnull(), 'mean_position'] = random.choice([8,9,10,11,12,13,14,15,16,17,18,19,20])


to_pred_reduced['mean_position_rank'] = to_pred_reduced.groupby("srch_id")['mean_position'].rank("dense", ascending=True)
to_pred_reduced['mean_position_rank_n'] = to_pred_reduced['mean_position_rank']/to_pred_reduced['srch_results_count']


to_pred_reduced['srch_mean_avg_children'] =  to_pred_reduced.groupby('srch_id')['hotel_avg_children'].transform('mean') 
to_pred_reduced['srch_mean_avg_children'] = to_pred_reduced['srch_mean_avg_children'].replace(np.nan,  0)
to_pred_reduced['srch_mean_avg_adult'] =  to_pred_reduced.groupby('srch_id')['hotel_avg_adult'].transform('mean')
to_pred_reduced['srch_mean_avg_adult'] = to_pred_reduced['srch_mean_avg_adult'].replace(np.nan,  0)
to_pred_reduced['srch_mean_avg_stay'] =  to_pred_reduced.groupby('srch_id')['hotel_avg_stay'].transform('mean')
to_pred_reduced['srch_mean_avg_stay'] = to_pred_reduced['srch_mean_avg_stay'].replace(np.nan,  0)
to_pred_reduced['srch_mean_avg_room_count'] =  to_pred_reduced.groupby('srch_id')['hotel_avg_room_count'].transform('mean')
to_pred_reduced['srch_mean_avg_room_count'] = to_pred_reduced['srch_mean_avg_room_count'].replace(np.nan,  0)


to_pred_reduced.loc[to_pred_reduced['hotel_avg_children'].isnull(), "hotel_avg_children"] = to_pred_reduced['srch_mean_avg_children']
to_pred_reduced.loc[to_pred_reduced['hotel_avg_adult'].isnull(), "hotel_avg_adult"] = to_pred_reduced['srch_mean_avg_adult']
to_pred_reduced.loc[to_pred_reduced['hotel_avg_stay'].isnull(), "hotel_avg_stay"] = to_pred_reduced['srch_mean_avg_stay']
to_pred_reduced.loc[to_pred_reduced['hotel_avg_room_count'].isnull(), "hotel_avg_room_count"] = to_pred_reduced['srch_mean_avg_room_count']

to_pred_reduced['diff_avg_children'] = (to_pred_reduced['srch_children_count'] - to_pred_reduced['hotel_avg_children'])
to_pred_reduced['diff_avg_adult'] = (to_pred_reduced['srch_adults_count'] - to_pred_reduced['hotel_avg_adult'])/to_pred_reduced['srch_adults_count']
to_pred_reduced['diff_avg_stay'] = (to_pred_reduced['srch_length_of_stay'] - to_pred_reduced['hotel_avg_stay'])/to_pred_reduced['srch_length_of_stay']
to_pred_reduced['diff_avg_room_count'] = (to_pred_reduced['srch_room_count'] - to_pred_reduced['hotel_avg_room_count'])/to_pred_reduced['srch_room_count']

to_pred_reduced['diff_avg_children1']=to_pred_reduced['diff_avg_children'].abs()
to_pred_reduced['diff_avg_adult1']=to_pred_reduced['diff_avg_adult'].abs()
to_pred_reduced['diff_avg_stay1']=to_pred_reduced['diff_avg_stay'].abs()
to_pred_reduced['diff_avg_room_count1']=to_pred_reduced['diff_avg_room_count'].abs()

to_pred_reduced['rank_child_match'] = to_pred_reduced.groupby("srch_id")["diff_avg_children1"].rank("dense", ascending=True)
to_pred_reduced['rank_adult_match'] = to_pred_reduced.groupby("srch_id")['diff_avg_adult1'].rank("dense", ascending=True)
to_pred_reduced['rank_stay_match'] = to_pred_reduced.groupby("srch_id")['diff_avg_stay1'].rank("dense", ascending=True)
to_pred_reduced['rank_room_match'] = to_pred_reduced.groupby("srch_id")['diff_avg_room_count1'].rank("dense", ascending=True)


to_pred_reduced['rank_child_match_n'] = to_pred_reduced['rank_child_match']/to_pred_reduced['srch_results_count']
to_pred_reduced['rank_adult_match_n'] = to_pred_reduced['rank_adult_match']/to_pred_reduced['srch_results_count']
to_pred_reduced['rank_stay_match_n'] = to_pred_reduced['rank_stay_match']/to_pred_reduced['srch_results_count']
to_pred_reduced['rank_room_match_n'] = to_pred_reduced['rank_room_match']/to_pred_reduced['srch_results_count']
to_pred_reduced['rank2_n'] = to_pred_reduced['rank_child_match_n'] + to_pred_reduced['rank_adult_match_n'] + to_pred_reduced['rank_stay_match_n'] + to_pred_reduced['rank_room_match_n']

to_pred_reduced['rank3_n'] = to_pred_reduced['rank_distance_n'] + to_pred_reduced['rank_price_n'] + to_pred_reduced['rank_stay_match_n'] + to_pred_reduced['rank_room_match_n']
#to_pred_reduced['estimated_position'] = to_pred_reduced.groupby("prop_id")["position"].transform('mean')

to_pred_reduced['rank2'] = to_pred_reduced['rank_child_match'] + to_pred_reduced['rank_adult_match'] + to_pred_reduced['rank_stay_match'] + to_pred_reduced['mean_position_rank']
to_pred_reduced['rank3'] = to_pred_reduced['rank_distance'] + to_pred_reduced['rank_price'] + to_pred_reduced['rank_stay_match'] + to_pred_reduced['mean_position_rank_n']



to_pred_reduced['rank4'] = to_pred_reduced['rank2'] + to_pred_reduced['rank'] + to_pred_reduced['rank_distance'] + to_pred_reduced['mean_position_rank'] 
#to_pred_reduced['estimated_position'] = to_pred_reduced.groupby("prop_id")["position"].transform('mean')
to_pred_reduced['rank5'] = to_pred_reduced['rank_child_match'] + to_pred_reduced['rank_adult_match'] + to_pred_reduced['rank_stay_match'] + to_pred_reduced['rank_room_match'] +to_pred_reduced['price_prev_rank']



In [47]:
#to_pred_reduced[['price_usd', 'price_usdnorm_bysrch_id']].isnull().mean() * 100

In [48]:
#to_pred_reduced = normalize_features(to_pred_reduced,group_key="srch_id",target_column="price_usd",take_log10=True,)

In [49]:
#to_pred_reduced['price_usd'] = to_pred_reduced['price_usd'].fillna(0)


In [50]:
###droppping unused columns 
#del hotel_click_stats
#del to_pred

# to_drop = ['site_id', 'date_time', 'visitor_location_country_id', 'prop_country_id','price_bin', 'prop_brand_bool', 'orig_destination_distance', 'prop_location_score2'
#           ,'price_bin', 'max_price', 'srch_query_affinity_score','visitor_hist_starrating', 'visitor_hist_adr_usd', 'min_price', 'mean_price', 'mean_price_bin_review','mean_price_bin_star', 'bought'] 



to_drop = ['site_id', 'visitor_location_country_id', 'prop_country_id',  'orig_destination_distance'
          , 'max_price','min_price', 'mean_price', 'mean_price_bin_review','mean_price_bin_star'] 






#del to_pred_reduced[ 'visitor_hist_adr_usd']

#to_drop = ['min_price', 'mean_price', 'mean_price_bin_review','mean_price_bin_star', 'bought',  'weekday', 'week_of_year', 'hour', 'time_epoch', 'month']
to_pred1= to_pred_reduced.drop(to_drop,  axis=1)

In [51]:

#del to_pred_reduced


In [52]:
#replacing nulls and nans

#to_pred1.replace([np.inf, -np.inf], 0, inplace=True)

#to_pred1[['hotel_qrtr_click_prob', 'hotel_qrtr_buy_prob', 'hotel_srch_sesn_buy_prob', 'hotel_srch_sesn_click_prob']] = to_pred1[['hotel_qrtr_click_prob', 'hotel_qrtr_buy_prob', 'hotel_srch_sesn_buy_prob', 'hotel_srch_sesn_click_prob']].replace(np.nan, 0)

#to_pred1=to_pred1.dropna(axis=1,how="any")
#msn.bar(to_pred1,figsize=(18,3), color='red')

In [53]:
# del to_pred1['price_usd']
# del to_pred1[ 'price_bin']
# del to_pred1['diff_location2_score']
# del to_pred1["'diff_location2_score"]
# del to_pred1['median_orig_dest_distance']
# del to_pred1['mean_orig_dest_distance']
# del to_pred1['diff_orig_dest_distance']
# del to_pred1['diff_orig_dest_distance1']
              
#               , 'price_bin', 'diff_location2_score', "'diff_location2_score",
#        'median_orig_dest_distance', 'mean_orig_dest_distance',
#        'diff_orig_dest_distance', 'diff_orig_dest_distance1']]

In [54]:
to_pred1.columns

Index(['srch_id', 'prop_id', 'prop_starrating', 'prop_review_score',
       'prop_location_score1', 'prop_location_score2', 'price_usd',
       'promotion_flag', 'srch_destination_id', 'srch_length_of_stay',
       ...
       'rank_child_match_n', 'rank_adult_match_n', 'rank_stay_match_n',
       'rank_room_match_n', 'rank2_n', 'rank3_n', 'rank2', 'rank3', 'rank4',
       'rank5'],
      dtype='object', length=101)

In [55]:
##ordering df and selecting predictors
#to_pred1 = to_pred1[['srch_id', 'srch_destination_id', 'prop_id','prop_starrating', 'prop_location_score1','prop_log_historical_price','promotion_flag','srch_length_of_stay','srch_booking_window','srch_adults_count','srch_children_count','srch_room_count','srch_saturday_night_bool','diff_max_price','diff_min_price','diff_mean_price','hotel_show_prob', 'prop_review_score','price_review_value','rank_child_match','rank_adult_match','rank_stay_match','rank_room_match','mean_review_srch','diff_mean_review','mean_star_srch','diff_mean_star','rank_price','rank_location','rank_review','rank_star','rank_location2','rank','rank_distance','rank2', 'rank3','location_size','random_bool']]
to_pred1 = to_pred1[['srch_id', 'prop_id', 'promotion_flag',
     'random_bool',  'diff_max_price',
       'diff_mean_price',
       'diff_mean_review',
       'diff_mean_star',  
       'location_size',  'srch_results_count','prop_starrating',
       'srch_%','rank_distance','rank_price',
       'rank_location', 'rank_review',
          'rank_star', 'rank_location2', 'prop_location_score2',
      'rank' ,'hotel_show_prob', 'show_feature', 'hotel_bought_bool',
         'mean_position', 'hotel_buy_prob',
         'rank_child_match', 'rank_adult_match', 'rank_stay_match',

         'rank_room_match', 'rank2',
       'rank3', 'price_usdnorm_bysrch_id','mean_position_rank_n','mean_position_rank','diff_med_pp','price_prev_rank','price_prev_abs_rank','rank4', 'rank5','hotel_buy_share','price_usdnorm_byprop_id', 'loc_%buy']]

# to_pred1 = to_pred1[['srch_id', 'prop_id', 'promotion_flag',
#       'srch_length_of_stay', 'srch_booking_window',
       
#        'srch_saturday_night_bool', 'random_bool',  'diff_max_price',
#        'diff_mean_price',
#        'diff_mean_review',
#        'diff_mean_star',  
#        'rank_distance', 'location_size',  'srch_results_count',
#        'srch_%', 'rank_distance_n','rank_price_n',
#         'rank_location_n',  'rank_review_n',
#         'rank_star_n',  'rank_location2_n',
#        'rank_n', 'hotel_show_prob', 'show_feature', 'hotel_bought_bool',
#        'hotel_click_bool', 'mean_position', 'hotel_avg_children',
#        'hotel_avg_adult', 'hotel_avg_stay', 'hotel_avg_room_count',
     
#        'diff_avg_children', 'diff_avg_adult',
#        'diff_avg_stay', 'diff_avg_room_count', 
#         'rank_child_match_n', 
#        'rank_adult_match_n',  'rank_stay_match_n',
#         'rank_room_match_n',  'rank2_n', 
#        'rank3_n', 'price_usdnorm_bysrch_id','mean_position_rank_n','rank4_n']]

#to_pred1 = to_pred1[['srch_destination_id', 'prop_starrating', 'prop_location_score1','prop_log_historical_price','promotion_flag','srch_length_of_stay','srch_booking_window','srch_adults_count','srch_children_count','srch_room_count','srch_saturday_night_bool', 'hotel_click_prob', 'hotel_buy_prob','diff_max_price','diff_min_price','diff_mean_price','hotel_show_prob', 'prop_review_score','price_review_value', 'hotel_avg_children','hotel_avg_adult','hotel_avg_stay','hotel_avg_room_count','random_bool','qrtr','hotel_qrtr_click_prob', 'hotel_qrtr_buy_prob', 'hotel_srch_sesn_buy_prob','hotel_srch_sesn_click_prob']]

In [56]:


#gc.collect()

#model = pickle.load(open(os.path.join(output_dir, "model.dat"), "rb"))

#test_data = test_data.copy()
#test_data = input_estimated_position(test_data, srch_id_dest_id_dict)

test_data_srch_id_prop_id = to_pred1[["srch_id", "prop_id"]]

to_pred1 = to_pred1.drop(['prop_id', "srch_id" ],axis =1)
#test_data = remove_columns(test_data)

#categorical_features_numbers = get_categorical_column(test_data)

print("Predicting on train set with columns: {}".format(to_pred1.columns.values))
#kwargs = {}
#kwargs = {"categorical_feature": categorical_features_numbers}

predictions = model.predict(to_pred1)
test_data_srch_id_prop_id["prediction"] = predictions
del to_pred1
#gc.collect()


Predicting on train set with columns: ['promotion_flag' 'random_bool' 'diff_max_price' 'diff_mean_price'
 'diff_mean_review' 'diff_mean_star' 'location_size' 'srch_results_count'
 'prop_starrating' 'srch_%' 'rank_distance' 'rank_price' 'rank_location'
 'rank_review' 'rank_star' 'rank_location2' 'prop_location_score2' 'rank'
 'hotel_show_prob' 'show_feature' 'hotel_bought_bool' 'mean_position'
 'hotel_buy_prob' 'rank_child_match' 'rank_adult_match' 'rank_stay_match'
 'rank_room_match' 'rank2' 'rank3' 'price_usdnorm_bysrch_id'
 'mean_position_rank_n' 'mean_position_rank' 'diff_med_pp'
 'price_prev_rank' 'price_prev_abs_rank' 'rank4' 'rank5' 'hotel_buy_share'
 'price_usdnorm_byprop_id' 'loc_%buy']


In [57]:
test_data_srch_id_prop_id = test_data_srch_id_prop_id.sort_values(['srch_id', 'prediction'],ascending = [True, False])
test_data_srch_id_prop_id1 = test_data_srch_id_prop_id.drop(['prediction'], axis = 1)
test_data_srch_id_prop_id1.to_csv('submission2.csv', index=False)

test_data_srch_id_prop_id1.head(5)

,srch_id,prop_id
4833,1,99484
1329,1,34263
948,1,28181
2615,1,61934
4253,1,90385


In [58]:
# ## saving serch_id and prod_id for use in submission file 

# ids = to_pred1[['srch_id', 'prop_id']]

# to_drop = ['srch_id','prop_id']

# to_pred1 = to_pred1.drop(to_drop,  axis=1)

# ###### perdicitng outcome using model

# y_pred=l[1][0].predict(to_pred1)

# print(y_pred)

# y_pred[:5]

In [59]:

# ###creating submission file

# predicted= pd.DataFrame(y_pred, columns=['predicted'])

# df = pd.DataFrame(to_pred1)

# df['perdicted'] = predicted

# df[['srch_id', 'prop_id']] = ids[['srch_id', 'prop_id']]

# df = df[['srch_id', 'prop_id', 'perdicted','hotel_buy_prob']]

# ##ordering submission file 


# df = df.sort_values(['srch_id', 'perdicted', 'hotel_buy_prob'],
#               ascending = [True, False,False])
# df = df.drop(['perdicted', 'hotel_buy_prob'], axis = 1)
# df.to_csv('submission.csv', index=False)

# df.head(5)